In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
import fileutils as fileutils
import gc
import psutil
import pickle
import fnmatch
import os
import time
import clfutils
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score
import new_alternate_single_svm as nalsvm

In [65]:


def feature_labels_fit(features, labels):
    label_name = str(labels.columns[labels.columns.str.contains(pat='label')].values[0])
    features_df = nalsvm.hmm_features_df(
        features)  # features data-frame - this just unbundles the features into a dataframe
    # lets get all the features in order now#
    if features_df.isnull().values.all() == True:
        print('HMM Features Problematic') #do i need a counter here?
        market_features_df = nalsvm.CreateMarketFeatures(
            nalsvm.CreateMarketFeatures(
                nalsvm.CreateMarketFeatures(df=nalsvm.CreateMarketFeatures(df=labels).ma_spread_duration())
                    .ma_spread()).chaikin_mf()).obv_calc()  # market features dataframe
        df_concat = market_features_df.dropna()
        df = df_concat[df_concat[label_name].notna()]
        df_final = df.drop(columns=['TradedPrice', 'Duration', 'TradedTime', 'ReturnTradedPrice', \
                                    'Volume', label_name])
    else:
        market_features_df = nalsvm.CreateMarketFeatures(
            nalsvm.CreateMarketFeatures(
                nalsvm.CreateMarketFeatures(df=nalsvm.CreateMarketFeatures(df=labels).ma_spread_duration())
                    .ma_spread()).chaikin_mf()).obv_calc()  # market features dataframe
        df_concat = pd.DataFrame(pd.concat([features_df, market_features_df], axis=1, sort='False').dropna())
        df = df_concat[df_concat[label_name].notna()]
        df_final = df.drop(columns=['TradedPrice', 'Duration', 'TradedTime', 'ReturnTradedPrice', \
                                    'Volume', label_name])

    if df_final.shape[0] < 10:
        print(' the ratio of classes is too low. try another label permutation')
    else:
        X_ = MinMaxScaler().fit_transform(df_final)
        y_labels = df[df.columns[df.columns.str.contains(pat='label')]].iloc[:, 0]

        return X_, y_labels
    
def hmm_features_df(features):
    features_df = nalsvm.hmm_features_df(
        features)  # features data-frame - this just unbundles the features into a dataframe
    return features_df

def logmemoryusage(msg):
    # function to log memory usage
    process = psutil.Process(os.getpid())
    print('Memory usage at %s is %smb.' % (msg, process.memory_info().rss / 1000 / 1000))


def unpickle_csv(pickled_csv):
    with open(pickled_csv, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        p = u.load()

    return p


def ticker_features_labels(file_joint_locations):
    # input a joint location file that contains both features and labels and returns one
    labels = pd.read_csv(file_joint_locations[1])
    features = unpickle_csv(file_joint_locations[0])
    return features, labels


def hmm_features_df(features_tuple):
    return pd.concat([features_tuple[0], features_tuple[1], \
                      features_tuple[2], features_tuple[3]], axis=1, sort=False)


def evaluate_predictions(y_true, y_preds):
    """
    Performs evaluation comparison on y_true labels vs. y_pred labels
    on a classification.
    """
    accuracy = accuracy_score(y_true, y_preds)
    precision = precision_score(y_true, y_preds, average='weighted')
    recall = recall_score(y_true, y_preds, average='weighted')
    f1 = f1_score(y_true, y_preds, average='weighted')
    metric_dict = {"accuracy": round(accuracy, 2),
                   "precision": round(precision, 2),
                   "recall": round(recall, 2),
                   "f1": round(f1, 2)}
    print(f"Acc: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1:.2f}")

    return metric_dict


########################################################################################################################
# paths
dataDrive = '/media/ak/My Passport/Data/FinDataReal/'  # also labels location folder
jointLocationsPickleFolder = os.path.join(dataDrive, 'JointLocationsDicts')
extPath = '/media/ak/My Passport/ExperimentData'
featuresPath = "/".join((extPath, 'features'))  # path with features

# Labels
labels_location_folder = fileutils.data_path  # this is the folder where all the labels are saved

labels_pickle_files = sorted([s for s in os.listdir(labels_location_folder) if ('LabelsAlternate') in s if
                              not ('.pkl') in s])  # these are all the dicts that we have alternate labels for.
# labels_pickle_files: these are all the dicts that we have alternate labels for.

symbols = [f for f in [s for s in os.listdir(labels_location_folder) if '.L' in s if '_Features' not in s] if
           ('.L_A' or '_Features') not in f]  # from all
forwardDates = [f for f in os.listdir(dataDrive) if 'ForwardDates' in f]
svcModels = [g for g in os.listdir(dataDrive) if '_SingleKernelSVC' in g]



In [106]:
svcModels

['AAL.L_LabelsAlternateFour_SingleKernelSVC.pkl',
 'BARC.L_LabelsAlternateFour_SingleKernelSVC.pkl',
 'BARC.L_LabelsAlternateOne_SingleKernelSVC.pkl',
 'BARC.L_LabelsAlternateThree_SingleKernelSVC.pkl',
 'BATS.L_LabelsAlternateFour_SingleKernelSVC.pkl',
 'AAL.L_LabelsAlternateOne_SingleKernelSVC.pkl']

In [107]:
symbol = 'BARC.L'
symbolSVCModels = [g for g in svcModels if str(symbol) in g]
symbolSVCModels

['BARC.L_LabelsAlternateFour_SingleKernelSVC.pkl',
 'BARC.L_LabelsAlternateOne_SingleKernelSVC.pkl',
 'BARC.L_LabelsAlternateThree_SingleKernelSVC.pkl']

In [108]:
jointLocationsPickleFolder

'/media/ak/My Passport/Data/FinDataReal/JointLocationsDicts'

In [109]:
alternate_label_idx = 2
jointLocsSymbols = list(np.unique([f.split("_")[0] for f in os.listdir(jointLocationsPickleFolder)]))


In [110]:
symbol_idx = jointLocsSymbols.index(symbol)

data_cls = nalsvm.AlternateLabelFeaturesLoader(path_main=dataDrive, symbol=symbol,
                                               alternate_label_idx=alternate_label_idx,
                                               jointLocationsPickleInput=jointLocationsPickleFolder)

In [111]:

jointLocationsDictionary = (data_cls.load_pickled_in_filename(data_cls.return_pickled_dict()))
joint_keys = data_cls.joint_loc_pickle_keys(data_cls.return_pickled_dict())


  
joint_keys == list(jointLocationsDictionary.keys())

True

In [112]:

# jointLocsSymbols[symbol_idx]--> this returns the symbol.

print('which label I am using ',labels_pickle_files[alternate_label_idx])  # 
logmemoryusage("Before garbage collect")
gc.collect()  # continue

which label I am using  LabelsAlternateOne
Memory usage at Before garbage collect is 301.162496mb.


2942

### SVC Model Location and Alternate Labels ###############################

In [113]:


symbolSVCModelLocationIndexedLabel = \
    [f for f in symbolSVCModels if str(labels_pickle_files[alternate_label_idx]) in f][0]
print(symbolSVCModelLocationIndexedLabel)
symbolSVCModelLocation = os.path.join(dataDrive, symbolSVCModelLocationIndexedLabel)
print(symbolSVCModelLocation)


BARC.L_LabelsAlternateOne_SingleKernelSVC.pkl
/media/ak/My Passport/Data/FinDataReal/BARC.L_LabelsAlternateOne_SingleKernelSVC.pkl


In [114]:
symbolSVCModels


['BARC.L_LabelsAlternateFour_SingleKernelSVC.pkl',
 'BARC.L_LabelsAlternateOne_SingleKernelSVC.pkl',
 'BARC.L_LabelsAlternateThree_SingleKernelSVC.pkl']

In [115]:
alternate_label = symbolSVCModelLocation.split("_")[1]
print(alternate_label) # do I really need this?

LabelsAlternateOne


In [116]:

######### MODELS #########################################################
models = unpickle_csv(symbolSVCModelLocation)
modelDates = list(models[list(models.keys())[0]].keys())

modelDateIdx = 0  # <-- this is fit-date in sample that corresponds to the labels date too
print(modelDates[modelDateIdx])

20170117


In [117]:

svc = models[str(symbol)][modelDates[modelDateIdx]]['SVC']

In [118]:


start = time.time()
if symbol in jointLocsSymbols:
    print('ok to go')
    print(symbol, ' and labels ', labels_pickle_files[alternate_label_idx])
    features, labels = ticker_features_labels(jointLocationsDictionary[modelDates[modelDateIdx]])
    X_fit, y_fit = feature_labels_fit(features, labels)
    fitted_model = svc.fit(X_fit, y_fit)



ok to go
BARC.L  and labels  LabelsAlternateOne
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   53.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.4min finished


In [119]:
symbolForwardDates = data_cls.forwardDates(joint_keys, modelDates[modelDateIdx]) # out of sample dates
symbolForwardDates

['20170118',
 '20170119',
 '20170120',
 '20170123',
 '20170124',
 '20170125',
 '20170126',
 '20170127',
 '20170130',
 '20170131',
 '20170801',
 '20170802',
 '20170803',
 '20170804',
 '20170807',
 '20170808',
 '20170809',
 '20170810',
 '20170811',
 '20170814',
 '20170815',
 '20170816',
 '20170817',
 '20170818',
 '20170821',
 '20170822',
 '20170823',
 '20170824',
 '20170825',
 '20170829',
 '20170830',
 '20170831',
 '20170901',
 '20170904',
 '20170905',
 '20170906',
 '20170907',
 '20170908',
 '20170911',
 '20170912',
 '20170913',
 '20170914',
 '20170915',
 '20170918',
 '20170919',
 '20170920',
 '20170921',
 '20170922',
 '20170925',
 '20170926',
 '20170927',
 '20170928',
 '20170929']

In [66]:

for forward_date_idx, _ in enumerate(symbolForwardDates):
    features_oos, labels_oos = ticker_features_labels(
        jointLocationsDictionary[symbolForwardDates[forward_date_idx]])
    if hmm_features_df(features_oos).isnull().values.all()==True:
        print('Problem')
    else:
        print(labels.value_counts())
        X_true, y_true = feature_labels_fit(features_oos, labels_oos)
        y_pred = fitted_model.predict(X_true)
        print(evaluate_predictions(y_true, y_pred))

end = time.time()
print(f'it took {end - start} seconds!')


Problem
479     1
550     1
5164    1
Name: label_EndPo__window_9__thres_arbitrary__0.1, dtype: int64


/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 49.87%
Precision: 0.77
Recall: 0.50
F1 score: 0.58
{'accuracy': 0.5, 'precision': 0.77, 'recall': 0.5, 'f1': 0.58}
Problem
479     1
550     1
5164    1
Name: label_EndPo__window_9__thres_arbitrary__0.1, dtype: int64


/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 44.53%
Precision: 0.79
Recall: 0.45
F1 score: 0.53
{'accuracy': 0.45, 'precision': 0.79, 'recall': 0.45, 'f1': 0.53}
479     1
550     1
5164    1
Name: label_EndPo__window_9__thres_arbitrary__0.1, dtype: int64


...

/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 28.57%
Precision: 0.90
Recall: 0.29
F1 score: 0.42
{'accuracy': 0.29, 'precision': 0.9, 'recall': 0.29, 'f1': 0.42}
Problem
Problem
479     1
550     1
5164    1
Name: label_EndPo__window_9__thres_arbitrary__0.1, dtype: int64


/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 30.59%
Precision: 0.86
Recall: 0.31
F1 score: 0.42
{'accuracy': 0.31, 'precision': 0.86, 'recall': 0.31, 'f1': 0.42}
479     1
550     1
5164    1
Name: label_EndPo__window_9__thres_arbitrary__0.1, dtype: int64
Acc: 31.36%
Precision: 0.90
Recall: 0.31
F1 score: 0.43
{'accuracy': 0.31, 'precision': 0.9, 'recall': 0.31, 'f1': 0.43}
it took 2704.6839826107025 seconds!


/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
features, labels= ticker_features_labels(
        jointLocationsDictionary[symbolForwardDates[0]])